In [1]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

#Loads the VGG16 model 
model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

Using TensorFlow backend.


In [2]:
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " " + layer.__class__.__name__ + "\t" ,layer.trainable)

0 InputLayer	 False
1 Conv2D	 True
2 Conv2D	 True
3 MaxPooling2D	 True
4 Conv2D	 True
5 Conv2D	 True
6 MaxPooling2D	 True
7 Conv2D	 True
8 Conv2D	 True
9 Conv2D	 True
10 MaxPooling2D	 True
11 Conv2D	 True
12 Conv2D	 True
13 Conv2D	 True
14 MaxPooling2D	 True
15 Conv2D	 True
16 Conv2D	 True
17 Conv2D	 True
18 MaxPooling2D	 True


In [3]:
for layer in model.layers:
    layer.trainable = False
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " " + layer.__class__.__name__ + "\t" ,layer.trainable)

0 InputLayer	 False
1 Conv2D	 False
2 Conv2D	 False
3 MaxPooling2D	 False
4 Conv2D	 False
5 Conv2D	 False
6 MaxPooling2D	 False
7 Conv2D	 False
8 Conv2D	 False
9 Conv2D	 False
10 MaxPooling2D	 False
11 Conv2D	 False
12 Conv2D	 False
13 Conv2D	 False
14 MaxPooling2D	 False
15 Conv2D	 False
16 Conv2D	 False
17 Conv2D	 False
18 MaxPooling2D	 False


# joining Top and Bottom of models

In [4]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [5]:
model.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 16
FC_Head = addTopModel(model, num_classes)
modelnew = Model(inputs=model.input, outputs=FC_Head)

In [7]:
print(modelnew.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [8]:
from keras.preprocessing.image import ImageDataGenerator

In [9]:
#I haven't done data augmentation here because I already created images using a my own augmentation script
# It has variety of options if ypou want to check I have addded with this Python File

In [10]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="/root/mlopsproject/UntitledFolder/ImageCrop/",target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="/root/mlopsproject/UntitledFolder/Testing/", target_size=(224,224))

Found 626 images belonging to 16 classes.
Found 66 images belonging to 16 classes.


In [11]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [12]:
checkpoint = ModelCheckpoint("friends_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
modelnew.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 377
nb_validation_samples = 54
epochs = 3
batch_size = 1

history = modelnew.fit_generator(
    traindata,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = testdata)

modelnew.save("friends_vgg16.h5")

Epoch 1/6
20/20 [==============================] - 203s 10s/step - loss: 84.6154 - accuracy: 0.2508 - val_loss: 6.2269 - val_accuracy: 0.4848

Epoch 00001: val_loss improved from inf to 6.22688, saving model to friends_vgg.h5
Epoch 2/6
20/20 [==============================] - 168s 8s/step - loss: 2.1239 - accuracy: 0.6262 - val_loss: 0.9889 - val_accuracy: 0.3485

Epoch 00002: val_loss improved from 6.22688 to 0.98886, saving model to friends_vgg.h5
Epoch 3/6
20/20 [==============================] - 179s 9s/step - loss: 1.9712 - accuracy: 0.6390 - val_loss: 0.0000e+00 - val_accuracy: 0.3030

Epoch 00003: val_loss improved from 0.98886 to 0.00000, saving model to friends_vgg.h5
Epoch 4/6
20/20 [==============================] - 175s 9s/step - loss: 3.1019 - accuracy: 0.6693 - val_loss: 0.0167 - val_accuracy: 0.5000

Epoch 00004: val_loss did not improve from 0.00000
Epoch 5/6
20/20 [==============================] - 173s 9s/step - loss: 1.8744 - accuracy: 0.7300 - val_loss: 0.0000e+00 -

In [ ]:
import cv2
import numpy as np
input_im = cv2.imread('/root/mlopsproject/UntitledFolder/ImageCrop/Aman/IMG20200130145913_cropped_2.jpg')
input_original = input_im.copy()
input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)

input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
input_im = input_im / 255.
input_im = input_im.reshape(1,224,224,3) 

# Get Prediction
res = np.argmax(modelnew.predict(input_im, 1, verbose = 0), axis=1)
print(res)
# Show image with predicted class
cv2.imshow('PRED',input_original)
cv2.waitKey(0)
cv2.destroyAllWindows()

[9]


In [14]:
traindata.class_indices

{'Abhishek': 0,
 'Akshansh': 1,
 'Alok': 2,
 'Aman': 3,
 'Ankit': 4,
 'Dipali': 5,
 'Krish': 6,
 'Nityam': 7,
 'Rishu': 8,
 'Ritesh': 9,
 'Ritesh2': 10,
 'Sameer': 11,
 'Sarthak': 12,
 'Shivam': 13,
 'Shubham': 14,
 'Vikash': 15}